In [ ]:
%%capture
import sys
!{sys.executable} -m pip install --user requests

In [ ]:
import sys
print(sys.version)

In [ ]:
# import Crawler : crawler for one full website
# import Downloader: Remote download & storing to local storage
%run /notebooks/python3-kernel/libs/crawler-class.ipynb

# Initialization

## Initialize crawler

* Turn `debug=True` passing it as last parameter

In [ ]:
BASE_URL = "https://xxx"

# Aim to only retrieve URL & subpage
crawler = Crawler(BASE_URL)
soup = crawler.parseUrl(BASE_URL)

print(soup.title)
# print(soup.prettify())

#### Filter HTML tag and normalize names

In [ ]:
def pagenameFromTag(tag):
    return pagename(tag.text.strip())

def pagename(fullname):
    splitted_names = str(fullname).split("-", 1)
    name = splitted_names[1]
    name = name.strip().replace(" ", "-")
    return name.strip()

pagenameFromTag(soup.title)
pagename("Free Download GIFs - Get the best GIF")

## Initialize Downloader

Downloader help to :

* retrieve content
* organize & store them in local folder

In [ ]:
BASE_PATH="/data/crawler/" + pagenameFromTag(soup.title)

downloader = Downloader(BASE_PATH, BASE_URL)
downloader.getRootFolder()

# Crawl & Download content

In [ ]:
def downloadResourceSection(page_prefix, path, section, suffix, debug=False):
    downloadResource(page_prefix + path, path + "(" + section + ")" + suffix)

def downloadResource(page_url, path, debug=False):
    soup = crawler.loadUrls(page_url, "tag", "non", "existing")
    if (debug):
        print(soup.prettify())
    downloader.downloadAllTags(path, soup, "a", "data-size", "^1", "href", debug)
    downloader.downloadAllTags(path, soup, "img", "src", "^" + BASE_URL, None, debug)
    downloader.downloadAllTags(path, soup, "source", "src", "^" + BASE_URL, None, debug)
    downloader.downloadAllTags(path, soup, "video", "src", "^" + BASE_URL, None, debug)


In [ ]:
# TEST DOWNLOAD
downloadResourceSection(BASE_URL + "staff-details.html?id=", "1234", "category1", "suffix", True)

In [ ]:
# Load all URL found for 1 page
page_url = BASE_URL
capture_section = "index.php?section="
capture_page = "page.php?id="

soup = crawler.loadUrls(page_url, "a", "class", "^MenuLink-sc-1ftqwtj", None, True)
soup = crawler.loadUrlsFromLinks(page_url, "a", "href", capture_section, True)
print(crawler.getUniqueUrls())
# print(soup.prettify())

In [ ]:
def extract_name(full_url, base_url, debug=False):
    regex = "^" + re.escape(base_url) + "(.+)$"
    name_grp = re.search(regex, full_url)
    if (debug):
        print("Search regex:", regex, " response:", name_grp)
    if name_grp is not None:
        return name_grp.group(1)

extract_name("/base/section/abcd", "/base/page/", True) # Should not raise error
extract_name("/base/page/id_1234", "/base/page/", True)

extract_name("index.php?section=1", capture_section, True) # Should not raise error
extract_name("page.php?id=1234", capture_page, True)

In [ ]:
def build_url(*paths):
    return BASE_URL + "".join(paths)

build_url(capture_section, "123")

In [ ]:
capture_section = BASE_URL + capture_section
capture_page = BASE_URL + capture_page

# SECTION = "https://skip_url"
for section_url in crawler.getUniqueUrls():
    print("== Found section_url:", section_url)

    if Url(section_url) != Url(BASE_URL) : # and Url(section_url) != Url(SECTION):
        # Aim to downlaod image from subpage
        crawlerPage = Crawler(BASE_URL)
        soup = crawlerPage.parseUrl(section_url)

        section = extract_name(section_url, capture_section, True)
        soup = crawlerPage.loadUrlsFromLinks(section_url, "a", "href", capture_page)
        print("== Found section:", section, " unique_urls:", crawlerPage.getUniqueUrls())

        for page_url in crawlerPage.getUniqueUrls():
            item = extract_name(page_url, capture_page)
            if item is not None:
                if section is None:
                    path = item
                else:
                    path = item + "(" + section + ")"
                print("* Load path:", path)
                if not downloader.isExist(path):
                    downloadResource(page_url, path, debug=False)